In [1]:
import os


In [2]:
%pwd

'/Users/sanghyuk/myProjects/mlproject02/research'

In [3]:
os.chdir('../')
%pwd

'/Users/sanghyuk/myProjects/mlproject02'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([Path(config.root_dir)])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config
            

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [11]:



class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            
            logger.info(f"{filename} downloaded with the following headers {headers}")
        else:
            logger.info(f"{filename} already exists of size {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self) -> None:
        """
        zip_file_path: Path
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion = DataIngestion(config.get_data_ingestion_config())
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-10 10:45:13,100: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-10 10:45:13,104: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-10 10:45:13,105: INFO: common: Created directory at: artifacts]
[2024-07-10 10:45:13,106: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-07-10 10:45:27,989: INFO: 3030074557: artifacts/data_ingestion/data.zip downloaded with the following headers Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F2DE:2DA1E1:10AA61D:1207071:668EAC88
Accept-Ranges: bytes
Date: Wed, 10 Jul 2024 15:45:13 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdal2120026